### Installing libraries:

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00


### GPU Check

In [2]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import evaluate

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cpu":
    print("⚠️ WARNING: You are running on CPU. Go to Runtime > Change runtime type > Select T4 GPU.")

Using device: cuda


### Load the Dataset

In [11]:
# Configuration
MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-en-fr"
SOURCE_LANG = "en"
TARGET_LANG = "fr"

dataset = load_dataset("opus_books", f"{SOURCE_LANG}-{TARGET_LANG}")

# MODIFICATION: Take only the first 10k examples for faster training
small_train_dataset = dataset["train"].select(range(10000))

# Split the small dataset
split_datasets = small_train_dataset.train_test_split(test_size=0.2, seed=42)

print(split_datasets)
print("Example data:", split_datasets["train"][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 2000
    })
})
Example data: {'id': '6910', 'translation': {'en': 'Colonel Forster believed that more than a thousand pounds would be necessary to clear his expenses at Brighton.', 'fr': 'Le colonel Forster estimait qu’il faudrait plus de mille livres pour régler ses dépenses a Brighton.'}}


### Tokenization

In [12]:
tokenizer = MarianTokenizer.from_pretrained(MODEL_CHECKPOINT)

def preprocess_function(examples):
    inputs = [ex[SOURCE_LANG] for ex in examples["translation"]]
    targets = [ex[TARGET_LANG] for ex in examples["translation"]]

    # Tokenize inputs (English)
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Tokenize targets (French)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the function to the whole dataset
tokenized_datasets = split_datasets.map(preprocess_function, batched=True)

print("Tokenization complete.")

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenization complete.


### Metrics & Model Setup

In [13]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode generated predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in labels (ignored index) with pad token
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

# Load the model
model = MarianMTModel.from_pretrained(MODEL_CHECKPOINT).to(device)

### Training Configuration

In [14]:
batch_size = 16
model_name = MODEL_CHECKPOINT.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-en-to-fr",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2254458931.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


### Starting to train

In [15]:
print("Starting training...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Starting training...


Step,Training Loss
500,2.206200
1000,1.839800
1500,1.736600


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1500, training_loss=1.9275415852864584, metrics={'train_runtime': 293.9825, 'train_samples_per_second': 81.638, 'train_steps_per_second': 5.102, 'total_flos': 534475519819776.0, 'train_loss': 1.9275415852864584, 'epoch': 3.0})

### Saving and testing

In [17]:
# Save the model
save_path = "./my_translator_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")

# TEST THE MODEL
from transformers import pipeline

# Load your freshly trained model
translator = pipeline("translation", model=save_path, device=0 if device=="cuda" else -1)

text = "This is a great project for my portfolio."
translation = translator(text)

print(f"Input: {text}")
print(f"Translation: {translation[0]['translation_text']}")

Model saved to ./my_translator_model


Device set to use cuda:0


Input: This is a great project for my portfolio.
Translation: Cest un grand projet pour mon portefeuille.


### Download to your Computer

In [18]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive('my_translator_model', 'zip', './my_translator_model')

# Download it
files.download('my_translator_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>